# Clustering Crypto

In [624]:
# Initial imports
import pandas as pd
import hvplot.pandas
from pathlib import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Data Preprocessing

In [625]:
# Loade the cryptocurrencies data
file_path = Path("Resources/crypto_data.csv")
crypto_df = pd.read_csv(file_path, index_col=0)
crypto_df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [626]:
# Keep only cryptocurrencies that are on trading
crypto_df.drop(crypto_df[crypto_df['IsTrading'] == False].index, inplace=True) 
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [627]:
# Keep only cryptocurrencies with a working algorithm
crypto_df = crypto_df[crypto_df['TotalCoinsMined'].notna()]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000


In [628]:
# Remove the "IsTrading" column
crypto_df.drop(columns=["IsTrading"], inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [629]:
# Remove rows with at least 1 null value
crypto_df.dropna(thresh=1).head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [630]:
# Remove rows with cryptocurrencies without coins mined
crypto_df = crypto_df[crypto_df['TotalCoinsMined'].notna()]
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [631]:
# Fetch the cryptocurrencies names prior to drop them from crypto_df
list(crypto_df.CoinName)

['42 Coin',
 '404Coin',
 '808',
 'EliteCoin',
 'Bitcoin',
 'Ethereum',
 'Litecoin',
 'Dash',
 'Monero',
 'Ethereum Classic',
 'ZCash',
 'Bitshares',
 'DigiByte',
 'BitcoinDark',
 'Bitstake',
 'PayCoin',
 'ProsperCoin',
 'KoboCoin',
 'Spreadcoin',
 'ACoin',
 'Aero Coin',
 'ApexCoin',
 'ArchCoin',
 'Argentum',
 'Aurora Coin',
 'BetaCoin',
 'BlueCoin',
 'MyriadCoin',
 'MoonCoin',
 'ZetaCoin',
 'SexCoin',
 'Quatloo',
 'EnergyCoin',
 'QuarkCoin',
 'Riecoin',
 'Digitalcoin ',
 'LimeCoinX',
 'BitBar',
 'BitMark',
 'CashCoin',
 'Catcoin',
 'CryptoBullion',
 'CannaCoin',
 'CinderCoin',
 'CoolCoin',
 'CryptCoin',
 'CasinoCoin',
 'eMark',
 'Diamond',
 'Verge',
 'DarkCash',
 'DarkShibe',
 'DevCoin',
 'EarthCoin',
 'E-Gulden',
 'Elacoin',
 'Einsteinium',
 'Emerald',
 'Exclusive Coin',
 'Flappy Coin',
 'Fuel2Coin',
 'FIBRE',
 'FireRoosterCoin',
 'FlutterCoin',
 'Franko',
 'FeatherCoin',
 'GrandCoin',
 'GlobalCoin',
 'GoldCoin',
 'GlyphCoin',
 'Halcyon',
 'HoboNickels',
 'HunterCoin',
 'HyperStake',


In [632]:
# Remove the cryptocurrency name since it's not going to be used on the clustering algorithm
crypto_df.drop(columns=["CoinName"], inplace=True)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
808,SHA-256,PoW/PoS,0.000000e+00,0
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000


In [657]:
# Create dummies variables for text features
crypto_encoded = pd.get_dummies(crypto_df, columns=["Algorithm", "ProofType"])
crypto_encoded.head()



,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
808,0.000000e+00,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [658]:
file_path = Path("Resources/crypto_data_encoded.csv")
crypto_encoded.to_csv(file_path, index=False)

In [659]:
# Standardize data
data_scaler = StandardScaler()
data_scaler.fit(crypto_encoded)

StandardScaler(copy=True, with_mean=True, with_std=True)

### Reducing Dimensions Using PCA

In [687]:
# Use PCA to reduce dimension to 3 principal components
crypto_scaled = StandardScaler().fit_transform(crypto_encoded)
print(crypto_scaled[0:5])
pca = PCA(n_components=3)

[[-0.10282804 -0.03823841 -0.03823596 -0.03823596 -0.03823596 -0.03823596
  -0.05411338 -0.07664017 -0.03823596 -0.05411338 -0.05411338 -0.03823596
  -0.03823596 -0.18216065 -0.05411338 -0.03823596 -0.03823596 -0.08574929
  -0.03823596 -0.10160947 -0.06632365 -0.03823596 -0.03823596 -0.1642757
  -0.03823596 -0.03823596 -0.13908716 -0.03823596 -0.03823596 -0.07664017
  -0.03823596 -0.03823596 -0.03823596 -0.03823596 -0.06632365 -0.03823596
  -0.07664017 -0.08574929 -0.07664017 -0.03823596 -0.03823596 -0.12775161
  -0.1335313  -0.13908716 -0.03823596 -0.05411338 -0.03823596 -0.06632365
  -0.1689039  -0.03823596 -0.03823596 -0.03823596 -0.07664017 -0.17342199
  -0.33468341 -0.03823596 -0.08574929 -0.06632365 -0.05411338 -0.03823596
   1.42042992 -0.06632365 -0.03823596 -0.03823596 -0.06632365 -0.06632365
  -0.03823596 -0.03823596 -0.03823596 -0.03823596 -0.03823596 -0.03823596
  -0.03823596 -0.41586681 -0.03823596 -0.19054822 -0.03823596 -0.10870529
  -0.07664017 -0.09400279 -0.03823596 -

In [ ]:
# Create a DataFrame with the principal components data
df_crypto_pca = pd.DataFrame(
    data=crypto_scaled, 
    columns=["principal component 1", "principal component 2", "principal component 3" ],
    index = crypto_df.index
)
df_crypto_pca.head()

### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [1]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range ok k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_iris_pca)
    inertia.append(km.inertia_)


# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")


NameError: name 'KMeans' is not defined

Running K-Means with `k=4`

In [ ]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)
# Fit the model
model.fit(df_crypto_pca)
# Predict clusters
predictions = model.predict(df_crypto_pca)
# Create a new DataFrame including predicted clusters and cryptocurrencies features
df_crypto_pca["Predcted", "Feature"] = model.labels_
df_crypto_pca.head()

### Visualizing Results

#### 3D-Scatter with Clusters

In [ ]:
# Create a 3D-Scatter with the PCA data and the clusters
df_crypto_pca.hvplot.scatter(
    x="principal component 1",
    y="principal component 2",
    hover_cols=["Predicted"],
    by="Predicted",
    width = 800
)
fig.update_layout(legend=dict(x=0,y=1))
fig.show()

#### Table of Tradable Cryptocurrencies

In [ ]:
# Table with tradable cryptos
df_crypto_pca.insert(0,'CoinName', coin_names)
df_crypto_pca.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined'], width=800)

In [ ]:
# Print the total number of tradable cryptocurrencies
print (f'Total of {df_crypto_pca.count()} tradable cryptocurrencies)

#### Scatter Plot with Tradable Cryptocurrencies

In [ ]:
# Scale data to create the scatter plot


In [ ]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
